# Imports

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
from keras_preprocessing import image
from tensorflow import keras
from matplotlib import pyplot as plt
import mediapipe as mp

# Loading the CNN models

In [2]:
rclassifier = keras.models.load_model('righthandmodel.h5')
lclassifier = keras.models.load_model('lefthandmodel.h5')
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y"]

# Loading the LSTM Model

In [3]:
mp_holistic = mp.solutions.holistic#Holistic Model
mp_drawing = mp.solutions.drawing_utils#Drawing Utilities

In [4]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#Color Coversion
    image.flags.writeable = False                 #Image no longer Writeable
    results = model.process(image)                #Make Prediction
    image.flags.writeable = True                  #Image Writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#Color Conversion
    return image,results
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [5]:
# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])
model = keras.models.load_model('action.h5')

# Running Program

In [6]:
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y"]

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2)

offset = 20
imgSize = 300

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    if hands:
        for hand in hands:
            x,y,w,h = hand['bbox']
            if hand["type"]=="Right":
                classifier = rclassifier
            else:
                classifier = lclassifier

            imgWhite = np.ones((imgSize,imgSize, 3),np.uint8)*255
            imgCrop = img[y-offset:y+h+offset, x-offset:x+w+offset]

            aspectRatio = h/w
            try:
                if aspectRatio < 1:
                    k = imgSize/w
                    hCal = math.ceil(k*h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    hGap = math.ceil((imgSize-hCal)/2)
                    imgWhite[hGap:hCal+hGap,:] = imgResize
                    test_image = image.img_to_array(imgWhite)
                    test_image = np.expand_dims(test_image, axis=0)
                    result = classifier.predict(test_image)
                    cv2.rectangle(img, (x-offset,y+h+offset), (x+w+offset,y+h+50), (0,0,0), -1)
                    cv2.putText(img, ' '.join(labels[np.argmax(result[0])]), (x+10,y+h+45), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    k = imgSize/h
                    wCal = math.ceil(k*w)
                    imgResize = cv2.resize(imgCrop, (wCal,imgSize))
                    wGap = math.ceil((imgSize-wCal)/2)
                    imgWhite[:,wGap:wCal+wGap] = imgResize
                    test_image = image.img_to_array(imgWhite)
                    test_image = np.expand_dims(test_image, axis=0)
                    result = classifier.predict(test_image)
                    cv2.rectangle(img, (x-offset,y+h+offset), (x+w+offset,y+h+50), (0,0,0), -1)
                    cv2.putText(img, ' '.join(labels[np.argmax(result[0])]), (x+10,y+h+45), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            except:
                pass
    cv2.imshow("Maurj Feed", img)
    wKey = cv2.waitKey(1)
    if wKey & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [27]:
cap.release()
cv2.destroyAllWindows()

In [7]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2)

offset = 20
imgSize = 300

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        success, img = cap.read()
        #LSTM
        img, results = mediapipe_detection(img, holistic)
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            sentence = sentence[-1:]
        #CNN
        hands, img = detector.findHands(img)
        if hands:
            for hand in hands:
                x,y,w,h = hand['bbox']
                if hand["type"]=="Right":
                    classifier = rclassifier
                else:
                    classifier = lclassifier

                imgWhite = np.ones((imgSize,imgSize, 3),np.uint8)*255
                imgCrop = img[y-offset:y+h+offset, x-offset:x+w+offset]

                aspectRatio = h/w
                try:
                    if aspectRatio < 1:
                        k = imgSize/w
                        hCal = math.ceil(k*h)
                        imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                        hGap = math.ceil((imgSize-hCal)/2)
                        imgWhite[hGap:hCal+hGap,:] = imgResize
                        test_image = image.img_to_array(imgWhite)
                        test_image = np.expand_dims(test_image, axis=0)
                        result = classifier.predict(test_image)
                        cv2.rectangle(img, (x-offset,y+h+offset), (x+w+offset,y+h+50), (0,0,0), -1)
                        cv2.putText(img, ' '.join(labels[np.argmax(result[0])]), (x+10,y+h+45), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    else:
                        k = imgSize/h
                        wCal = math.ceil(k*w)
                        imgResize = cv2.resize(imgCrop, (wCal,imgSize))
                        wGap = math.ceil((imgSize-wCal)/2)
                        imgWhite[:,wGap:wCal+wGap] = imgResize
                        test_image = image.img_to_array(imgWhite)
                        test_image = np.expand_dims(test_image, axis=0)
                        result = classifier.predict(test_image)
                        cv2.rectangle(img, (x-offset,y+h+offset), (x+w+offset,y+h+50), (0,0,0), -1)
                        cv2.putText(img, ' '.join(labels[np.argmax(result[0])]), (x+10,y+h+45), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                except:
                    pass
        cv2.putText(img, 'LSTM Prediction', (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(img, ' '.join(sentence), (3,90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        cv2.imshow("Maurj Feed", img)
        wKey = cv2.waitKey(1)
        if wKey & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()